In [1]:
import numpy
from keras import models
from keras import layers, losses
from numpy import loadtxt
from keras import optimizers
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras import regularizers

from keras import layers
from keras import models

from keras import backend as K

Using TensorFlow backend.


In [2]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [4]:
metrics = ['accuracy', precision_m, recall_m]

model.compile(loss='binary_crossentropy',
            optimizer=optimizers.RMSprop(lr=1e-4),
            metrics= metrics)

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(r'../05/train',
                                                     target_size=(150, 150),# this is resize
                                                     batch_size=16,
                                                     class_mode='binary')
validation_generator = test_datagen.flow_from_directory(r'../05/validation',
                                                         target_size=(150, 150),
                                                         batch_size=16,
                                                         class_mode='binary')
test_generator = test_datagen.flow_from_directory(r'../05/test',
                                                  target_size=(150, 150),
                                                  batch_size=16,
                                                  class_mode='binary')
history = model.fit_generator(train_generator,
                              steps_per_epoch=10,
                              epochs=10,
                              validation_data = validation_generator,
                              validation_steps = 4,
                              )

Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/10
 2/10 [=====>........................] - ETA: 3s - loss: 0.6888 - accuracy: 0.5357 - precision_m: 0.5182 - recall_m: 0.5556

/Users/MondayMeoW/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:804: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/Users/MondayMeoW/anaconda3/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


10/10 [==============================] - 4s 416ms/step - loss: 0.6884 - accuracy: 0.5608 - precision_m: 0.5228 - recall_m: 0.6385 - val_loss: 0.6803 - val_accuracy: 0.6500 - val_precision_m: 0.5625 - val_recall_m: 0.9410
Epoch 2/10
10/10 [==============================] - 4s 357ms/step - loss: 0.6847 - accuracy: 0.5329 - precision_m: 0.5605 - recall_m: 0.6063 - val_loss: 0.5344 - val_accuracy: 0.5000 - val_precision_m: 0.5469 - val_recall_m: 1.0000
Epoch 3/10
10/10 [==============================] - 3s 346ms/step - loss: 0.6577 - accuracy: 0.6216 - precision_m: 0.6630 - recall_m: 0.8163 - val_loss: 0.5636 - val_accuracy: 0.7500 - val_precision_m: 0.9000 - val_recall_m: 0.6190
Epoch 4/10
10/10 [==============================] - 4s 377ms/step - loss: 0.5875 - accuracy: 0.6908 - precision_m: 0.6579 - recall_m: 0.6169 - val_loss: 0.9318 - val_accuracy: 0.5000 - val_precision_m: 0.5000 - val_recall_m: 1.0000
Epoch 5/10
10/10 [==============================] - 4s 367ms/step - loss: 0.6393 - 

In [6]:
print('\n----------------------------------------------MODEL 1----------------------------------------------\n')
print('\ntrain results:\n')
loss, accuracy, precision, recall = model.evaluate_generator(train_generator, verbose=0, steps = len(train_generator))
print("loss={:.2f}, accuracy={:.2f}, precision={:.2f}, recall={:.2f}".format(loss, accuracy, precision, recall))
print('\ntest results:\n')
loss, accuracy, precision, recall = model.evaluate_generator(test_generator, verbose=0, steps = len(test_generator))
print("loss={:.2f}, accuracy={:.2f}, precision={:.2f}, recall={:.2f}".format(loss, accuracy, precision, recall))



----------------------------------------------MODEL 3----------------------------------------------


train results:

loss=0.82, accuracy=0.75, precision=0.90, recall=0.61

test results:

loss=0.55, accuracy=0.90, precision=0.94, recall=0.75


In [7]:
model2 = models.Sequential()
model2.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Flatten())
model2.add(layers.Dense(512, activation='relu'))
model2.add(layers.Dense(32, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy',
                optimizer=optimizers.RMSprop(lr=1e-4),
                metrics= metrics)
history = model2.fit_generator(train_generator,
                              steps_per_epoch=10,
                              epochs=10,
                              validation_data = validation_generator,
                              validation_steps = 4,
                              )

Epoch 1/10


/Users/MondayMeoW/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:804: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


 1/10 [==>...........................] - ETA: 9s - loss: 0.6891 - accuracy: 0.6250 - precision_m: 0.6364 - recall_m: 0.7778

/Users/MondayMeoW/anaconda3/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


10/10 [==============================] - 6s 621ms/step - loss: 0.7017 - accuracy: 0.5000 - precision_m: 0.4210 - recall_m: 0.6806 - val_loss: 0.6802 - val_accuracy: 0.7000 - val_precision_m: 0.6875 - val_recall_m: 0.3988
Epoch 2/10
10/10 [==============================] - 6s 602ms/step - loss: 0.6839 - accuracy: 0.5132 - precision_m: 0.6750 - recall_m: 0.7431 - val_loss: 0.7093 - val_accuracy: 0.7500 - val_precision_m: 0.3818 - val_recall_m: 0.4444
Epoch 3/10
10/10 [==============================] - 6s 572ms/step - loss: 0.6514 - accuracy: 0.6486 - precision_m: 0.6660 - recall_m: 0.8071 - val_loss: 0.5608 - val_accuracy: 0.6500 - val_precision_m: 0.6929 - val_recall_m: 1.0000
Epoch 4/10
10/10 [==============================] - 6s 649ms/step - loss: 0.6220 - accuracy: 0.6579 - precision_m: 0.7114 - recall_m: 0.6390 - val_loss: 1.0648 - val_accuracy: 0.5000 - val_precision_m: 0.4531 - val_recall_m: 0.7500
Epoch 5/10
10/10 [==============================] - 8s 837ms/step - loss: 0.5875 - 

In [8]:
print('\n----------------------------------------------MODEL 2----------------------------------------------\n')
print('\ntrain results:\n')
loss, accuracy, precision, recall = model2.evaluate_generator(train_generator, verbose=0, steps = len(train_generator))
print("loss={:.2f}, accuracy={:.2f}, precision={:.2f}, recall={:.2f}".format(loss, accuracy, precision, recall))
print('\ntest results:\n')
loss, accuracy, precision, recall = model2.evaluate_generator(test_generator, verbose=0, steps = len(test_generator))
print("loss={:.2f}, accuracy={:.2f}, precision={:.2f}, recall={:.2f}".format(loss, accuracy, precision, recall))


----------------------------------------------MODEL 2----------------------------------------------


train results:

loss=0.40, accuracy=0.75, precision=0.78, recall=0.71

test results:

loss=0.36, accuracy=0.90, precision=0.90, recall=1.00


In [9]:
model3 = models.Sequential()
model3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Flatten())
model3.add(layers.Dropout(0.4))
model3.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model3.add(layers.Dense(32, activation='relu'))
model3.add(layers.Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy',
                optimizer=optimizers.RMSprop(lr=1e-4),
                metrics= metrics)
history = model3.fit_generator(train_generator,
                              steps_per_epoch=10,
                              epochs=10,
                              validation_data = validation_generator,
                              validation_steps = 4,
                              )

Epoch 1/10


/Users/MondayMeoW/anaconda3/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


 2/10 [=====>........................] - ETA: 4s - loss: 1.8211 - accuracy: 0.3571 - precision_m: 0.4583 - recall_m: 0.5455

/Users/MondayMeoW/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:804: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


10/10 [==============================] - 5s 461ms/step - loss: 1.6643 - accuracy: 0.5000 - precision_m: 0.3542 - recall_m: 0.4313 - val_loss: 1.6171 - val_accuracy: 0.5000 - val_precision_m: 0.5000 - val_recall_m: 1.0000
Epoch 2/10
10/10 [==============================] - 4s 440ms/step - loss: 1.4907 - accuracy: 0.6014 - precision_m: 0.5896 - recall_m: 0.5333 - val_loss: 1.6027 - val_accuracy: 0.6500 - val_precision_m: 0.5144 - val_recall_m: 1.0000
Epoch 3/10
10/10 [==============================] - 4s 408ms/step - loss: 1.4328 - accuracy: 0.6014 - precision_m: 0.6669 - recall_m: 0.7224 - val_loss: 1.1899 - val_accuracy: 0.7500 - val_precision_m: 0.8125 - val_recall_m: 0.6190
Epoch 4/10
10/10 [==============================] - 4s 416ms/step - loss: 1.3397 - accuracy: 0.6382 - precision_m: 0.6997 - recall_m: 0.6841 - val_loss: 1.0888 - val_accuracy: 0.7500 - val_precision_m: 0.7504 - val_recall_m: 1.0000
Epoch 5/10
10/10 [==============================] - 4s 400ms/step - loss: 1.2776 - 

In [10]:
print('\n----------------------------------------------MODEL 3----------------------------------------------\n')
print('\ntrain results:\n')
loss, accuracy, precision, recall = model3.evaluate_generator(train_generator, verbose=0, steps = len(train_generator))
print("loss={:.2f}, accuracy={:.2f}, precision={:.2f}, recall={:.2f}".format(loss, accuracy, precision, recall))
print('\ntest results:\n')
loss, accuracy, precision, recall = model3.evaluate_generator(test_generator, verbose=0, steps = len(test_generator))
print("loss={:.2f}, accuracy={:.2f}, precision={:.2f}, recall={:.2f}".format(loss, accuracy, precision, recall))


----------------------------------------------MODEL 1----------------------------------------------


train results:

loss=1.05, accuracy=0.68, precision=0.83, recall=0.45

test results:

loss=1.10, accuracy=0.90, precision=1.00, recall=0.69
